In [245]:
querys = pd.read_csv('../datas/investing_query.csv')

In [260]:
datetime.datetime()



TypeError: an integer is required (got type str)

In [265]:
today

210802

In [267]:
update_date = querys.values[0][2].split(' ')[0]
update_date = int(update_date.split('-')[0]) * 10000 + int(update_date.split('-')[1]) * 100 + int(update_date.split('-')[2])

update_date

20211026

In [258]:
import datetime

In [239]:
querys.drop(querys[querys['symbol']=='OGN'].index[0], inplace=True)

In [242]:
querys.to_csv('../datas/investing_query.csv', index=False)

In [201]:
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import warnings     
warnings.filterwarnings(action='ignore')    
import user

In [232]:
querys['symbol'].values

array(['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP',
       'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP',
       'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS',
       'APA', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T',
       'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC',
       'BK', 'BAX', 'BDX', 'BRKB', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA',
       'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BFB', 'CHRW',
       'COG', 'CDNS', 'CZR', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR',
       'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN',
       'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI',
       'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS',
       'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ',
   

In [189]:
querys = pd.read_csv('../datas/investing_query.csv')

investing = pd.read_csv
today = datetime.now().year % 100 * 10000 + \
    datetime.now().month * 100 + datetime.now().day
df = []
crawling_investings = []
a = 0
for row in querys.values:
    symbol = row[0]
    query = row[1]
    update_date = row[2]
    [sales, shared, update_date]  = crawling_investing(query)
    time.sleep(0.5)
    dfs1, dfs2, dfs3, dfs4 = crawling_investing2(query)
    #dfs1 = crawling_investing2(query)
    for k in [sales, shared, update_date]:
        dfs1.append(k)
        dfs2.append(k)
        dfs3.append(k)
        dfs4.append(k)
    crawling_investings.append(dfs1)
    crawling_investings.append(dfs2)
    crawling_investings.append(dfs3)
    crawling_investings.append(dfs4)
    querys['update_date'][querys['symbol'] == symbol] = update_date

In [190]:
crawling_investings

[['MMM',
  210630,
  8950.0,
  4242.0,
  1971.0,
  1524.0,
  34730000000.0,
  578638253,
  211025],
 ['MMM',
  210331,
  8851.0,
  4327.0,
  1983.0,
  1624.0,
  34730000000.0,
  578638253,
  211025],
 ['MMM',
  201231,
  8583.0,
  4246.0,
  1839.0,
  1389.0,
  34730000000.0,
  578638253,
  211025],
 ['MMM',
  200930,
  8350.0,
  4047.0,
  1909.0,
  1413.0,
  34730000000.0,
  578638253,
  211025],
 ['ABT',
  210630,
  10223.0,
  5276.0,
  1342.0,
  1189.0,
  40230000000.0,
  1776820148,
  211019],
 ['ABT',
  210331,
  10456.0,
  6056.0,
  2109.0,
  1793.0,
  40230000000.0,
  1776820148,
  211019],
 ['ABT',
  201231,
  10701.0,
  6237.0,
  2437.0,
  2162.0,
  40230000000.0,
  1776820148,
  211019],
 ['ABT',
  200930,
  8853.0,
  4890.0,
  1545.0,
  1232.0,
  40230000000.0,
  1776820148,
  211019],
 ['ABBV',
  210331,
  13010.0,
  8818.0,
  4103.0,
  3553.0,
  50200000000.0,
  1766222336,
  211028],
 ['ABBV',
  201231,
  13858.0,
  9174.0,
  3753.0,
  36.0,
  50200000000.0,
  1766222336,


In [194]:
investing_cr= pd.DataFrame(crawling_investings,columns=["Symbol", 'update_date', 'querter_sales','profit','operating' ,'real_profit','sales' ,'share' ,'next_update_date'])

In [197]:
investing_cr = investing_cr.append(pd.DataFrame(['US500'], columns=['Symbol']))

In [198]:
investing_cr

,Symbol,update_date,querter_sales,profit,operating,real_profit,sales,share,next_update_date
0,MMM,210630.0,8950.0,4242.0,1971.0,1524.0,3.473000e+10,5.786383e+08,211025.0
1,MMM,210331.0,8851.0,4327.0,1983.0,1624.0,3.473000e+10,5.786383e+08,211025.0
2,MMM,201231.0,8583.0,4246.0,1839.0,1389.0,3.473000e+10,5.786383e+08,211025.0
3,MMM,200930.0,8350.0,4047.0,1909.0,1413.0,3.473000e+10,5.786383e+08,211025.0
4,ABT,210630.0,10223.0,5276.0,1342.0,1189.0,4.023000e+10,1.776820e+09,211019.0
...,...,...,...,...,...,...,...,...,...
2016,ZTS,210331.0,1871.0,1328.0,687.0,559.0,7.010000e+09,4.749734e+08,210804.0
2017,ZTS,201231.0,1807.0,1209.0,420.0,359.0,7.010000e+09,4.749734e+08,210804.0
2018,ZTS,200930.0,1786.0,1243.0,597.0,479.0,7.010000e+09,4.749734e+08,210804.0
2019,ZTS,200630.0,1548.0,1102.0,482.0,377.0,7.010000e+09,4.749734e+08,210804.0


In [202]:
connect_datas = {
    'host': user.host,
    'user': user.user,
    'passwd': user.pw,
    'db': user.db,
    'charset': 'utf8'
}
db = MySQLdb.connect(**connect_datas) 

client = create_engine(f'mysql://{user.user}:{user.pw}@{user.host}/{user.db}?charset=utf8',encoding="utf-8")
conn = client.connect()

In [204]:
# # cursor객체 생성후 쿼리 실행
investing_cr.to_sql(name='Company_low',con=client,if_exists='append',index=False)

IntegrityError: (pymysql.err.IntegrityError) (1452, 'Cannot add or update a child row: a foreign key constraint fails (`US_Stock`.`Company_low`, CONSTRAINT `Company_low_ibfk_1` FOREIGN KEY (`Symbol`) REFERENCES `company` (`Symbol`))')
[SQL: INSERT INTO `Company_low` (`Symbol`, update_date, querter_sales, profit, operating, real_profit, sales, share, next_update_date) VALUES (%(Symbol)s, %(update_date)s, %(querter_sales)s, %(profit)s, %(operating)s, %(real_profit)s, %(sales)s, %(share)s, %(next_update_date)s)]
[parameters: ({'Symbol': 'MMM', 'update_date': 210630.0, 'querter_sales': 8950.0, 'profit': 4242.0, 'operating': 1971.0, 'real_profit': 1524.0, 'sales': 34730000000.0, 'share': 578638253.0, 'next_update_date': 211025.0}, {'Symbol': 'MMM', 'update_date': 210331.0, 'querter_sales': 8851.0, 'profit': 4327.0, 'operating': 1983.0, 'real_profit': 1624.0, 'sales': 34730000000.0, 'share': 578638253.0, 'next_update_date': 211025.0}, {'Symbol': 'MMM', 'update_date': 201231.0, 'querter_sales

In [226]:
b = investing_cr['Symbol'].unique()

In [ ]:
SELECT Symbol FROM US_Stock.company;

In [220]:
sql = '''
SELECT Symbol FROM US_Stock.company;
''' 
# # cursor객체 생성후 쿼리 실행
curs = db.cursor()
data1 = db_class.executeAll(sql)

In [218]:
db_class= MysqlModule.Database() 

In [217]:
import MysqlModule

In [225]:
a = pd.DataFrame(data1)['Symbol'].values

In [227]:
for data in a :
    if data not in b :
        print(data)

In [228]:
for data in b :
    if data not in a :
        print(data)

OGN


In [179]:
querys =pd.read_csv('../datas/investing_query.csv')

In [176]:
querys['update_date'] = 0

In [178]:
querys.to_csv('../datas/investing_query.csv', index=False)

In [1]:
import re
import requests
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import pandas as pd
import time
import sys
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import warnings     
warnings.filterwarnings(action='ignore')    
import user

def crawlingquery(symbol):
    url = f"https://investing.com/search/?q={symbol}"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    # 맞는 데이터 찾기
    dom = BeautifulSoup(response.content, "html.parser")
    datas = dom.find_all(class_="js-inner-all-results-quote-item row")
    investing_query = np.nan

    for i in range(len(datas)):
        flag = datas[i].find(class_="flag first")
        try:
            if re.findall('middle (USA)', str(flag))[0] != 'USA':
                continue
        except:
            continue

        investingsymbol = datas[i].find(class_="second").text
        if investingsymbol.upper() != symbol.upper():
            continue

        text = datas[i].find(class_="fourth").text
        if text[:5] != 'Stock':
            continue

        investing_query = re.findall(
            'equities\/([\S]+)', datas[i].get("href"))[0]
        return investing_query

In [2]:
import datetime
def crawling_investing(query):
    try:
        url = f"https://kr.investing.com/equities/{query}"
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(url, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        elements = dom.find_all(
            class_='flex justify-between border-b py-2 desktop:py-0.5')
        # time.sleep(0.5)
        for element in elements:
            try:
                shared = re.findall('발행주식수([0-9\,]+)', element.text)[0]
                shared = int(shared.replace(',', ''))
            except:
                pass

            try:
                sales = re.findall('매출([0-9\.A-Z]+)', element.text)[0]
                sales = float(sales[:-1]) * int(sales[-1].replace('B', str(10000*10000*10)).replace('M', str(10000*100)))
            except:
                pass

            try:
                data = re.findall('다음 수익일자([0-9년월일\s]+)', element.text)[0]
                year = int(re.findall('(2[12])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                date = int(re.findall('([0-9]{1,2})일', data)[0])
                update_date = year*10000 + month*100 + date
                update_date = datetime.datetime.strptime(str(update_date), '%y%m%d') + datetime.timedelta(days=1)
            except:
                continue
            
            
            
        return [sales, shared, update_date]

    except:
        return [np.nan, np.nan, np.nan]

In [188]:
def crawling_investing2(query):
    querys = pd.read_csv('../datas/investing_query.csv')
    symbol = querys['symbol'][querys['query'] == query].values[0]
    first_date = first_sales = first_profit = first_operating = first_realprofit = second_date = second_sales = second_profit = second_operating = second_realprofit = thirth_date = thirth_sales = thirth_profit = thirth_operating = thirth_realprofit = fourth_date = fourth_sales = fourth_profit = fourth_operating = fourth_realprofit = np.nan
    try:
        url = f"https://kr.investing.com/equities/{query}-financial-summary"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
        response = requests.get(url, headers=headers)
        dom = BeautifulSoup(response.content, "html.parser")
        dates = dom.select("#rsdiv > div > table > thead > tr > th")
        numbers = dom.select("#rsdiv > div > table > tbody > tr > td")
        # time.sleep(0.5)

        for date, i in zip(dates, range(len(dates))):
            if date.text == '기말:':
                data = re.findall('[0-9년월일\s]+', dates[i+1].text)[0]
                year = int(re.findall('(2[0-9])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                day = int(re.findall('([0-9]{1,2})일', data)[0])
                first_date = year*10000 + month*100 + day
                data = re.findall('[0-9년월일\s]+', dates[i+2].text)[0]
                year = int(re.findall('(2[0-9])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                day = int(re.findall('([0-9]{1,2})일', data)[0])
                second_date = year*10000 + month*100 + day
                data = re.findall('[0-9년월일\s]+', dates[i+3].text)[0]
                year = int(re.findall('(2[0-9])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                day = int(re.findall('([0-9]{1,2})일', data)[0])
                thirth_date = year*10000 + month*100 + day
                data = re.findall('[0-9년월일\s]+', dates[i+4].text)[0]
                year = int(re.findall('(2[0-9])년', data)[0])
                month = int(re.findall('([0-9]{1,2})월', data)[0])
                day = int(re.findall('([0-9]{1,2})일', data)[0])
                fourth_date = year*10000 + month*100 + day
                break

        for number, i in zip(numbers, range(len(numbers))):
            if number.text == '총매출':
                try:
                    first_sales = float(numbers[i+1].text)
                    second_sales = float(numbers[i+2].text)
                    thirth_sales = float(numbers[i+3].text)
                    fourth_sales = float(numbers[i+4].text)
                except:
                    first_sales = second_sales = thirth_sales = fourth_sales = np.nan
            elif number.text == '총 이익':
                try:
                    first_profit = float(numbers[i+1].text)
                    second_profit = float(numbers[i+2].text)
                    thirth_profit = float(numbers[i+3].text)
                    fourth_profit = float(numbers[i+4].text)
                except:
                    first_profit = second_profit = thirth_profit = fourth_profit = np.nan

            elif number.text == '영업 이익':
                try:
                    first_operating = float(numbers[i+1].text)
                    second_operating = float(numbers[i+2].text)
                    thirth_operating = float(numbers[i+3].text)
                    fourth_operating = float(numbers[i+4].text)
                except:
                    first_operating = second_operating = thirth_operating = fourth_operating = np.nan

            elif number.text == '순이익':
                try:
                    first_realprofit = float(numbers[i+1].text)
                    second_realprofit = float(numbers[i+2].text)
                    thirth_realprofit = float(numbers[i+3].text)
                    fourth_realprofit = float(numbers[i+4].text)
                except:
                    first_realprofit = second_realprofit = thirth_realprofit = fourth_realprofit = np.nan
    
#        return first_date, first_sales, first_profit, first_operating, first_realprofit
        dfs1 = [symbol, first_date, first_sales,
                first_profit, first_operating, first_realprofit]
        dfs2 = [symbol, second_date, second_sales,
                second_profit, second_operating, second_realprofit]
        dfs3 = [symbol, thirth_date, thirth_sales,
                thirth_profit, thirth_operating, thirth_realprofit]
        dfs4 = [symbol, fourth_date, fourth_sales,
                fourth_profit, fourth_operating, fourth_realprofit]

    except:
#        return np.nan, np.nan, np.nan, np.nan, np.nan
        dfs1 = [symbol, np.nan, np.nan, np.nan, np.nan, np.nan]
        dfs2 = [symbol, np.nan, np.nan, np.nan, np.nan, np.nan]
        dfs3 = [symbol, np.nan, np.nan, np.nan, np.nan, np.nan]
        dfs4 = [symbol, np.nan, np.nan, np.nan, np.nan, np.nan]

    return dfs1, dfs2, dfs3, dfs4

In [165]:
import sys
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests
import time
import re
#from module import crawlingquery, crawling_investing, crawling_investing2

querys = pd.read_csv('../datas/investing_query.csv')

investing = pd.read_csv
today = datetime.now().year % 100 * 10000 + \
    datetime.now().month * 100 + datetime.now().day
df = []
crawling_investings = []
a = 0
for row in querys.values:
    symbol = row[0]
    query = row[1]
    update_date = row[2]
    url = f"https://kr.investing.com/equities/{query}-news"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
    response = requests.get(url, headers=headers)
    if response.status_code >= 300:
        a = 1
        query = crawlingquery(symbol)
        querys['investing_query'][querys['symbol'] == symbol] = query
        url = f"https://kr.investing.com/equities/{query}-news"
        response = requests.get(url, headers=headers)
        if response.status_code >= 300:
            print(f'{symbol}의 서버가 응답하지 않습니다. 에러 코드 {response.status_code}')

    dom = BeautifulSoup(response.content, "html.parser")
    datas = dom.find_all(class_="textDiv")

    for i in range(len(datas)):
        item = datas[i]
        try:
            text = item.select_one('.date').text
            date = re.findall('[0-9]{0,2} 시간 전', text)
        except:
            continue
        title = item.select_one(".title").get("title")
        subtitle = BeautifulSoup(item.select_one('p').text, "html.parser")
        if item.select_one(".title").get("href")[:8] != 'https://':
            link = 'https://kr.investing.com' + item.select_one(".title").get("href")
        else :
            link = item.select_one(".title").get("href")

        df.append({'symbol': symbol, 'title': title, 'subtitle': subtitle,"link": link})
    time.sleep(0.5)

    if today > update_date:
        [sales, shared, update_date]  = crawling_investing(query)
        time.sleep(0.5)
        dfs1, dfs2, dfs3, dfs4 = crawling_investing2(query)
        #dfs1 = crawling_investing2(query)
        for k in [sales, shared, update_date]:
            dfs1.append(k)
            dfs2.append(k)
            dfs3.append(k)
            dfs4.append(k)
        crawling_investings.append(dfs1)
        crawling_investings.append(dfs2)
        crawling_investings.append(dfs3)
        crawling_investings.append(dfs4)
        querys['update_date'][querys['symbol'] == symbol] = update_date
        a=1
        

#1. mysqldb 접속객체 세팅(연결하기)
connect_datas = {
    'host': user.host,
    'user': user.user,
    'passwd': user.pw,
    'db': user.db,
    'charset': 'utf8'
}
db = MySQLdb.connect(**connect_datas) 

client = create_engine(f'mysql://{user.user}:{user.pw}@{user.host}/{user.db}?charset=utf8',encoding="utf-8")
conn = client.connect()

##news 데이터 sql에 집어넣기
news = pd.DataFrame(df)
news.columns=['Symbol', 'title', 'subtitle', 'link']
# # MYsqldb로 테이블 생성하기
sql = '''CREATE TABLE News ( 
Symbol varchar(5) NOT NULL, 
title varchar(200),
subtitle varchar(200),
link varchar(200),
FOREIGN KEY (Symbol) REFERENCES company(Symbol)
) 
''' 
# # cursor객체 생성후 쿼리 실행
curs = db.cursor()
curs.execute(sql)


nonmaker = querys[['symbol']]
nonmaker.columns = ['Symbol']
for column in ['title', 'subtitle', 'link']:
    nonmaker[column] = np.nan
news = pd.concat([nonmaker, nonmaker, nonmaker, nonmaker, nonmaker, news])
news.to_sql(name='News',con=client,if_exists='append',index=False)

## investing 데이터 집어넣기
investing_cr= pd.DataFrame(crawling_investings,columns=["Symbol", 'update_date', 'querter_sales','profit','operating' ,'real_profit','sales' ,'share' ,'next_update_date'])

# # MYsqldb로 테이블 생성하기
sql = '''CREATE TABLE Company_low ( 
Symbol varchar(5) NOT NULL, 
update_date date, 
querter_sales double,
profit double,
operating double,
real_profit double,
sales double,
shared double,
next_update_date date,
FOREIGN KEY (Symbol) REFERENCES company(Symbol)
) 
''' 
# # cursor객체 생성후 쿼리 실행
curs = db.cursor()
curs.execute(sql)
investing_cr.to_sql(name='Company_low',con=client,if_exists='append',index=False)

#if a == 1:
#    querys.to_csv('../datas/investing_query.csv', index=False)

NameError: name 'MySQLdb' is not defined

In [166]:
df

[{'symbol': 'MMM',
  'title': '‘11언더파’ 이경훈, PGA 3M 오픈 공동 6위로 마무리',
  'subtitle': 이경훈(30·CJ대한통운)이 미국프로골프(PGA) 투어 3M 오픈(총상금 660만달러)을 공동 6위로 마무리했다. 이경훈은 26일(한국시간) 미국 미네소타 블레인의 TPC 투윈 ...,
  'link': 'https://invst.ly/vj8bn'},
 {'symbol': 'MMM',
  'title': "스카치테이프, 포스트잇…모두 '3M' 발명품 [히든業스토리]",
  'subtitle': 스카치테이프, 포스트잇. 모두 문구점에서 흔히 볼 수 있는 필수 사무용품들이다. 하지만 이 두 제품 모두 특정 기업이 한 기업이 특허권을 가진 '고유 브랜드'라는 사실을 아는 사람...,
  'link': 'https://invst.ly/vf3l4'},
 {'symbol': 'MMM',
  'title': "'포스트잇' 발명한 美화학자 스펜서 실버 타계 '향년 80세'",
  'subtitle': '포스트잇'을 발명한 화학자 스펜서 실버가 별세했다. 향년 80세. 14일(현지시간) 워싱턴포스트에 따르면 3M은 실버가 지난 8일 미국 미네소타주 세인트폴에 있는 자택에서 숨졌다...,
  'link': 'https://invst.ly/ut2bp'},
 {'symbol': 'MMM',
  'title': "'신월여의지하도로' 대형차 잇단 끼임 사고…16일부터 과태료 부과",
  'subtitle': 서울시가 지난달 16일 개통한 '신월여의지하도로'에서 중·대형차들의 무리한 진입으로 인해 개통 후 현재까지 20건의 착오진입이 발생함에 따라 방지대책을 강화에 나섰다. 4일 서울시...,
  'link': 'https://invst.ly/uob92'},
 {'symbol': 'MMM',
  'title': '경제 캘린더 - 이번 주 주목해야 할 주요 이슈 5가지',
  'subtitle': Investing.com - 다양한 실적과 연준의

In [170]:
news = pd.DataFrame(df)

In [174]:
news

,Symbol,title,subtitle,link
0,MMM,"‘11언더파’ 이경훈, PGA 3M 오픈 공동 6위로 마무리",[이경훈(30·CJ대한통운)이 미국프로골프(PGA) 투어 3M 오픈(총상금 660만...,https://invst.ly/vj8bn
1,MMM,"스카치테이프, 포스트잇…모두 '3M' 발명품 [히든業스토리]","[스카치테이프, 포스트잇. 모두 문구점에서 흔히 볼 수 있는 필수 사무용품들이다. ...",https://invst.ly/vf3l4
2,MMM,'포스트잇' 발명한 美화학자 스펜서 실버 타계 '향년 80세',['포스트잇'을 발명한 화학자 스펜서 실버가 별세했다. 향년 80세. 14일(현지시...,https://invst.ly/ut2bp
3,MMM,'신월여의지하도로' 대형차 잇단 끼임 사고…16일부터 과태료 부과,[서울시가 지난달 16일 개통한 '신월여의지하도로'에서 중·대형차들의 무리한 진입으...,https://invst.ly/uob92
4,MMM,경제 캘린더 - 이번 주 주목해야 할 주요 이슈 5가지,[Investing.com - 다양한 실적과 연준의 통화정책회의 개최로 변동성 높은...,https://kr.investing.com/news/economy/article-...
5,MMM,역사를 바꾼 30초·3m…인저뉴어티 비행 성공 막전막후,"[""30초, 3m의 비행이 역사를 바꿨다."" 인류가 비행체로 외계 행성을 누비는 시...",https://invst.ly/uithf
6,MMM,“3M-코카콜라 등 73개 기업서 청년인턴 뽑아요”,[서울시가 글로벌 기업이 참여하는 인턴 프로그램 ‘서울형 청년인턴 직무캠프’를 진행...,https://invst.ly/ud0-o
7,MMM,"오늘의 주요 3가지 이슈: 애플, 페이스북, 연준 금리",[By Liz Moyer\nInvesting.com -- 폐장 후 마이크로소프트 (...,https://kr.investing.com/news/stock-market-new...
8,MMM,"오늘의 주요 3가지 이슈: 고용 데이터, 므누신, 소매 실적",[By Christiana Sciaudone\nInvesting.com -- 경기부...,https://kr.investing.com/news/stock-market-new...
9,MMM,7월 10일 주목해야 할 시장의 주요 이슈 3가지,[By Christiana Sciaudone\nInvesting.com -- 나스닥...,https://kr.investing.com/news/stock-market-new...


In [152]:
querys

,symbol,query,update_date
0,MMM,3m-co,0
1,ABT,abbott-laboratories,0
2,ABBV,abbvie-inc,0
3,ABMD,abiomed,0
4,ACN,accenture-ltd,0
...,...,...,...
500,YUM,yum!-brands-inc,0
501,ZBRA,zebra-tech,0
502,ZBH,zimmer-hldgs,0
503,ZION,zions-bancorp,0


In [150]:
querys['update_date'] = 0

In [153]:
querys.to_csv('../datas/investing_query.csv', index=False)

In [154]:
pd.read_csv('../datas/investing_query.csv')

,symbol,query,update_date
0,MMM,3m-co,0
1,ABT,abbott-laboratories,0
2,ABBV,abbvie-inc,0
3,ABMD,abiomed,0
4,ACN,accenture-ltd,0
...,...,...,...
500,YUM,yum!-brands-inc,0
501,ZBRA,zebra-tech,0
502,ZBH,zimmer-hldgs,0
503,ZION,zions-bancorp,0


In [93]:
    if today > update_date:
        [sales, shared, update_date]  = crawling_investing(query)
        time.sleep(0.5)
        dfs1, dfs2, dfs3, dfs4 = crawling_investing2(query)
        #dfs1 = crawling_investing2(query)
        for k in [sales, shared, update_date]:
            dfs1.append(k)
            dfs2.append(k)
            dfs3.append(k)
            dfs4.append(k)
        crawling_investings.append(dfs1)
        crawling_investings.append(dfs2)
        crawling_investings.append(dfs3)
        crawling_investings.append(dfs4)
        querys['update_date'][querys['symbol'] == symbol] = update_date

In [160]:
import datetime

In [163]:
datetime.timedelta

datetime.datetime(2021, 8, 2, 4, 51, 52, 394521)

In [94]:
crawling_investing2(query)

(['MMM', 210630, 8950.0, 4242.0, 1971.0, 1524.0],
 ['MMM', 210331, 8851.0, 4327.0, 1983.0, 1624.0],
 ['MMM', 201231, 8583.0, 4246.0, 1839.0, 1389.0],
 ['MMM', 200930, 8350.0, 4047.0, 1909.0, 1413.0])

In [88]:
querys.head(20)

,symbol,query,update_date
0,MMM,3m-co,0.0
1,ABT,abbott-laboratories,211019.0
2,ABBV,abbvie-inc,211028.0
3,ABMD,abiomed,211103.0
4,ACN,accenture-ltd,210922.0
5,ATVI,activision-inc,211103.0
6,ADBE,adobe-sys-inc,210920.0
7,AMD,adv-micro-device,211026.0
8,AAP,advance-auto-parts,211115.0
9,AES,aes-corp,211110.0


In [73]:
        crawling_investings.append(dfs4)


In [74]:
crawling_investings

[['AES',
  210331,
  2635.0,
  664.0,
  166.0,
  -148.0,
  9960000000.0,
  666257334,
  210804],
 ['AES', 201231, 2560.0, 906.0, 771.0, 318.0, 9960000000.0, 666257334, 210804],
 ['AES',
  200930,
  2545.0,
  756.0,
  -292.0,
  -333.0,
  9960000000.0,
  666257334,
  210804],
 ['AES', 200630, 2217.0, 524.0, 423.0, -83.0, 9960000000.0, 666257334, 210804]]

In [60]:
dfs3

['AES', 200930, 2545.0, 756.0, -292.0, -333.0]

In [4]:
import sys
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import requests
import time
import re


In [23]:
url = "https://kr.investing.com/equities/apple-computer-inc-news"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"}
response = requests.get(url, headers=headers)
if response.status_code >= 300:
    a = 1
    query = crawlingquery(symbol)
    querys['investing_query'][querys['symbol'] == symbol] = query
    url = f"https://kr.investing.com/equities/{query}-news"
    response = requests.get(url, headers=headers)
    if response.status_code >= 300:
        print(f'{symbol}의 서버가 응답하지 않습니다. 에러 코드 {response.status_code}')

dom = BeautifulSoup(response.content, "html.parser")
datas = dom.find_all(class_="textDiv")


In [28]:
df = []
symbol = 'AAPL'
for i in range(len(datas)):
    item = datas[i]
    try:
        text = item.select_one('.date').text
        date = re.findall('[0-9]{0,2} 시간 전', text)
    except:
        continue
    title = item.select_one(".title").get("title")

    
    if item.select_one(".title").get("href")[:8] != 'https://':
        link = 'https://kr.investing.com' + item.select_one(".title").get("href")
    else :
        link = item.select_one(".title").get("href")
    #if date != []:
        #df.append({'symbol': symbol, 'title': title,"link": link, 'date': date})
    df.append({'symbol': symbol, 'title': title,"link": link, 'date': date})
time.sleep(0.5)


In [43]:
title = item.select_one(".title").get("title")

In [46]:
pd.DataFrame(df)

,symbol,title,link,date
0,AAPL,,https://kr.investing.com/news/economy/article-...,[1 시간 전]
1,AAPL,,https://kr.investing.com/news/economy/article-...,[1 시간 전]
2,AAPL,,https://kr.investing.com/news/economy/article-...,[21 시간 전]
3,AAPL,,https://kr.investing.com/news/economy/article-...,[22 시간 전]
4,AAPL,구글 직원,https://kr.investing.com/news/economy/article-...,[]
5,AAPL,,https://kr.investing.com/news/economy/article-...,[]
6,AAPL,"게임株, ‘써머랠리’ 시동…신작 흥행·中 판호 ‘쌍끌이’",https://kr.investing.com/news/economy/article-...,[]
7,AAPL,"삼성·하이닉스, 외국에 디지털세 낸다",https://kr.investing.com/news/economy/article-...,[]
8,AAPL,구글·애플에 큰 타격…한국 稅收도 늘어날 듯,https://kr.investing.com/news/economy/article-...,[]
9,AAPL,"[0702퇴근길] 국민연금, 2년 수익률 10% 안팎…고수익률 배경은 외 경제금융뉴스",https://kr.investing.com/news/stock-market-new...,[]


In [44]:
title

'[0702퇴근길] 국민연금, 2년 수익률 10% 안팎…고수익률 배경은 외 경제금융뉴스'

In [41]:
BeautifulSoup(item.select_one('p').text, "html.parser")

[인포스탁데일리=원세영 에디터]
네이버 오디오클립과 인포스탁데일리가 전해드리는 7월 2일 퇴근길 써머리입니다.
▲코스피 3281.78 마감 (-0.01%)
코스피 지수는 전일보다 ...

In [33]:
#leftColumn > div.mediumTitle1 > article:nth-child(1) > div.textDiv > p

'[0702퇴근길] 국민연금, 2년 수익률 10% 안팎…고수익률 배경은 외 경제금융뉴스'

In [14]:
datas

[<div class="textDiv">
 <a '화제'="" 2030년쯤엔…"="" [김익환의="" class="title" href="/news/economy/article-660309" title="구글 직원 " 美="" 글="" 블라인드="" 비트코인="" 외환·금융="" 워치]"="">구글 직원 "비트코인 2030년쯤엔…" 美 블라인드 글 '화제' [김익환의 외환·금융 워치]</a>
 </div>,
 <div class="textDiv">
 <a 2030="" [김익환의="" class="title" href="/news/economy/article-660350" title="" 꼽은="" 바엔="" 살="" 외환·금융="" 워치]"="" 직장인이="" 차라리"…美="" 테슬라="" 톱픽은="">"테슬라 살 바엔 차라리"…美 2030 직장인이 꼽은 톱픽은 [김익환의 외환·금융 워치]</a>
 </div>,
 <div class="textDiv">
 <a '상장폐지'="" [류은혁의="" class="title" href="/news/economy/article-660349" title="" 기업분석실]"="" 내="" 설마="" 알아야="" 주식도="" 징후="" 할="" 휴지될까?"…꼭="">"설마 내 주식도 휴지될까?"…꼭 알아야 할 '상장폐지' 징후 [류은혁의 기업분석실]</a>
 </div>,
 <div class="textDiv">
 <a class="title" href="/analysis/article-200436699" title="7월 증시, 하반기 증시의 시작 : 마음 단단히 먹고 투자해야만 한다">7월 증시, 하반기 증시의 시작 : 마음 단단히 먹고 투자해야만 한다</a>
 </div>,
 <div class="textDiv">
 <a class="title" href="/analysis/article-200436707" title="6월 시장 요약: 금리 인상 우려에도 탄탄한 실적 보인 미국 증시">6월 시장 요약: 금리 인상 우

In [40]:
df.s

,symbol,title,link,date
0,MMM,'포스트잇' 발명한 美화학자 스펜서 실버 타계 '향년 80세',https://kr.investing.comhttps://invst.ly/ut2bp,[]
1,MMM,'신월여의지하도로' 대형차 잇단 끼임 사고…16일부터 과태료 부과,https://kr.investing.comhttps://invst.ly/uob92,[]
2,MMM,경제 캘린더 - 이번 주 주목해야 할 주요 이슈 5가지,https://kr.investing.com/news/economy/article-...,[]
3,MMM,역사를 바꾼 30초·3m…인저뉴어티 비행 성공 막전막후,https://kr.investing.comhttps://invst.ly/uithf,[]
4,MMM,“3M-코카콜라 등 73개 기업서 청년인턴 뽑아요”,https://kr.investing.comhttps://invst.ly/ud0-o,[]
...,...,...,...,...
1626,YUM,오늘 주목해야 할 시장의 주요 이슈 3가지,https://kr.investing.com/news/stock-market-new...,[]
1627,ZBH,"짐머 바이오메트 Q1 실적, 수익 예상치를 넘어",https://kr.investing.com/news/article-631149,[]
1628,ZION,(BUZZ)美 은행주 하락...브렉시트에 美 금리인상 가능성 희박해져,https://kr.investing.com/news/economy-news/(bu...,[]
1629,ZTS,미국서 오랑우탄도 백신 맞아…동물용 첫 접종,https://kr.investing.comhttps://invst.ly/u1f4d,[]


In [24]:
querys

,symbol,query,update_date
0,MMM,3m-co,210726.0
1,ABT,abbott-laboratories,210720.0
2,ABBV,abbvie-inc,210722.0
3,ABMD,abiomed,210728.0
4,ACN,accenture-ltd,210922.0
...,...,...,...
500,YUM,yum!-brands-inc,210804.0
501,ZBRA,zebra-tech,210802.0
502,ZBH,zimmer-hldgs,210728.0
503,ZION,zions-bancorp,210718.0


In [28]:
crawling_investings = pd.DataFrame(crawling_investings)

In [29]:
crawling_investings.columns = ['symbol', 'update_date', 'querter_sales', 'profit', 'operating', 'real_profit', 'sales', 'shared', 'next_update_date']

In [31]:
crawling_investings.to_csv('../datas/crawling_investing.csv', index=False)

In [ ]:
a

In [ ]:
import pandas as pd
import FinanceDataReader as fdr
import numpy as np
from flask import *
app = Flask(__name__)

def stock_code(val):
    df_sp500 = fdr.StockListing('sp500')
    symbol = df_sp500[df_sp500['Symbol']==val]
    name = symbol.iloc[0,0]
    return name

stock_code('AAPL')
df = fdr.DataReader(stock_code('AAPL'), '2021')

@app.route('/')

@app.route('/home')
def home():
    return "hello worldddddd!"

@app.route('/stock')
def stock():
    return df


if __name__ == '__main__':
    app.run()

In [ ]:
def stock_code(val):
    df_sp500 = fdr.StockListing('sp500')
    symbol = df_sp500[df_sp500['Symbol']==val]
    name = symbol.iloc[0,0]
    return name

stock_code('AAPL')
df = fdr.DataReader(stock_code('AAPL'), '2021')
df


In [ ]:
app = Flask(__name__)

@app.route('/')

@app.route('/home')
def home():
    return "hello worldddddd!"

@app.route('/stock')
def stock_code(val):
    df_sp500 = fdr.StockListing('sp500')
    symbol = df_sp500[df_sp500['Symbol']==val]
    name = symbol.iloc[0,0]
    return name

def stock():
    stock_code('AAPL')
    df = fdr.DataReader(stock_code('AAPL'), '2021')
    df.to_html()


if __name__ == '__main__':
    app.run()

In [1]:
import sys

In [2]:
mod = sys.modules[__name__]

In [7]:
a = 'AAPL'

In [5]:
def getattr(mod, a)(t): 
    return t+1

SyntaxError: invalid syntax (<ipython-input-5-01179d2ff3cb>, line 1)

In [8]:
getattr(mod, a)

AttributeError: module '__main__' has no attribute 'AAPL'